In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
import os, torch, wandb, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login

In [2]:
base_model = "/notebooks/models/parser/models/Mixtral-8x7B-Instruct-v0.1-GPTQ/"

In [3]:
from auto_gptq import AutoGPTQForCausalLM
import auto_gptq
import transformers

In [4]:
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             )

In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

In [85]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=False,  # if using langchain set True
    task="text-generation",
    # we pass model parameters here too
    do_sample=True,  # set to False to use greedy decoding
    temperature=0.3,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.9,  # select from top tokens whose probability add up to 15%
    top_k=40,  # select from top 0 tokens (because zero, relies on top_p)
    typical_p=0.2,  # typical probability of a token being selected
    max_new_tokens=256,  # max number of tokens to generate in the output
    repetition_penalty=1.1,  # if output begins repeating increase,
    num_beams=2,
    length_penalty=0,
)

In [92]:
klayassistPrompt = """
You are a chabot in Klaytn crypto wallet for the Klaytn blockchain.
You can guide users to use the Klaytn blockchain.
You are representing the Klaytn blockchain and you are here to help users with their question.
If you do not know the answer to the question, you must respond with "I don't know".
DO NOT make up an answer, you must only respond with "I don't know" if you do not know the answer.
Do not answer questions that are not related to the Klaytn blockchain.

### Question: Is klaytn better than ethereum?
### Response: 

"""

In [93]:
import time

start = time.time()
res = generate_text(klayassistPrompt)
print(time.time() - start)
print(res[0]["generated_text"])

9.679630517959595
I am a Klaytn blockchain bot and I am not here to compare Klaytn with other blockchains. Klaytn has its own unique features and advantages, such as its high transaction speed and low fees. It is designed to support mass adoption of blockchain technology. However, whether Klaytn is "better" than Ethereum or any other blockchain depends on the specific use case and user needs. It is always best to evaluate each blockchain based on its own merits and how well it aligns with your goals.
